# Data modelling

## Preamble

In [88]:
import os
import sys
from importlib import reload

import requests
import json
import wikipedia
import selenium
import time
import urllib
import pycountry
import re
import tweepy

from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup

import unidecode

import pandas as pd

# add ./python to python path
sys.path.insert(0, '../python')

import data_acquisition

## List of reference PEP (heads of state and show business people)

In [5]:
wikipedia.set_lang("en")
page = wikipedia.page('List of current heads of state and government')

In [7]:
soup = BeautifulSoup(page.html(), "html.parser")

In [55]:
rows = soup.find('table', {'class': 'wikitable plainrowheaders'}).find_all('tr')


In [135]:
heads_of_state= {'country': [], 'firstname': [], 'lastname': [], 'country_code': []}
for row in rows[1:]:


    try:
        country_name = row.findAll('th')[0].text
        title_name = row.findAll('td')[0].text
        country_name = re.findall(r'\w+', country_name) #re.sub(r'\n', '', country_name)
        
        title = title_name.split('–')[0]
        name = title_name.split('–')[1]
        name = re.sub(r'\n', '', name)
        name = unidecode.unidecode(re.sub(r'\[.\]', '', name))
        
        firstname, lastname = name.split(' ')[1], ' '.join(name.split(' ')[2:])
        
       
        country_name = ' '.join(country_name)
        
        try:
            country_name = 'Gambia' if country_name == 'Gambia The' else country_name
            country_name = 'Guinea-Bissau' if country_name == 'Guinea Bissau' else country_name
            country_name = 'Iran, Islamic Republic of' if country_name == 'Iran' else country_name
            country_name = 'Côte d\'Ivoire' if country_name == 'Ivory Coast' else country_name
            country_name = 'Lao People\'s Democratic Republic' if country_name == 'Laos' else country_name
            country_name = 'Lao People\'s Democratic Republic' if country_name == 'Macedonia' else country_name





                
            country_code = pycountry.countries.get(name=country_name).alpha_3
        except:
            country_code = None
            
        heads_of_state['country'].append(country_name)
        heads_of_state['firstname'].append(firstname)
        heads_of_state['lastname'].append(lastname)
        heads_of_state['country_code'].append(country_code)

        
    except:

        heads_of_state['country'].append(None)
        heads_of_state['firstname'].append(None)
        heads_of_state['lastname'].append(None)
        heads_of_state['country_code'].append(None)

    
pd.DataFrame(heads_of_state).to_csv('heads_states.csv', index=False)

In [136]:
pycountry.countries.get(alpha_3='LAO')

Country(alpha_2='LA', alpha_3='LAO', name="Lao People's Democratic Republic", numeric='418')